### Equipo:
- Gutiérrez Pérez Gabriela G.
- Loeza Sebastián.
- Ramos García Luis Gerardo.
- Rico Mendoza Josué.

-----


## Extracción de mensajes de whatsApp:

In [1]:
import re
import csv

In [2]:
import re
import csv

def extractMessages(filePath):
    with open(filePath, 'r', encoding='utf-8') as file:
        lines = file.readlines()
    
    messages = []
    for line in lines:
        if re.match(r'^\d{2}/\d{2}/\d{2} \d{2}:\d{2} - Pika: ', line) \
           and '<Multimedia omitido>' not in line \
           and 'Se eliminó este mensaje.' not in line \
           and not re.search(r'(http|www)', line):
            
            # Limpiar el mensaje usando expresiones regulares
            mensaje_limpio = re.sub(r'^\d{2}/\d{2}/\d{2} \d{2}:\d{2} - Pika: ', '', line)
            mensaje_limpio = re.sub(r'<Se editó este mensaje\.>', '', mensaje_limpio).strip()
            messages.append(f"${mensaje_limpio}&")
    
    return messages

def saveMessagesToCsv(messages, outputFilePath):
    with open(outputFilePath, 'w', newline='', encoding='utf-8') as file:
        writer = csv.writer(file)
        writer.writerow(['Message'])
        for message in messages:
            writer.writerow([message])

# Ejemplo de uso:
# messages = extractMessages("ruta_del_archivo.txt")
# saveMessagesToCsv(messages, "archivo_resultado.csv")


In [3]:
chatFilePath = './MensajesJosue.txt'
outputCsvPath = './josue_messages.csv'
gabyMessages = extractMessages(chatFilePath)
saveMessagesToCsv(gabyMessages, outputCsvPath)
print(f'Mensajes de Josue guardados en: {outputCsvPath}')

FileNotFoundError: [Errno 2] No such file or directory: './MensajesJosue.txt'

In [ ]:
chatFilePath = './Chat de WhatsApp con Gaby.txt'
outputCsvPath = './gaby_messages.csv'
gabyMessages = extractMessages(chatFilePath)
saveMessagesToCsv(gabyMessages, outputCsvPath)
print(f'Mensajes de Gaby guardados en: {outputCsvPath}')

In [ ]:
chatFilePath = './MensajesPika.txt'
outputCsvPath = './pika_messages.csv'
gabyMessages = extractMessages(chatFilePath)
saveMessagesToCsv(gabyMessages, outputCsvPath)
print(f'Mensajes de Pika guardados en: {outputCsvPath}')

# Language model creation

In [4]:
import tkinter as tk
from tkinter import ttk, filedialog, messagebox
import pandas as pd
import nltk
from nltk.util import ngrams
from collections import Counter

In [5]:
# Tokenizar el corpus
def tokenizeCorpus(corpus):
    tokenizer = re.compile(r'\w+|[^\w\s]')  # Separar palabras y signos
    return [tokenizer.findall(text.lower()) for text in corpus]

# Generar n-gramas
def generateNgrams(tokenizedText, n):
    return list(ngrams(tokenizedText, n))

# Probabilidades de bigramas
def bigramProbabilities(tokenizedCorpus):
    bigrams = [ngram for text in tokenizedCorpus for ngram in generateNgrams(text, 2)]
    bigramFreq = Counter(bigrams)
    contextFreq = Counter([bigram[0] for bigram in bigrams])

    bigramData = [
        [bigram[0], bigram[1], freq, contextFreq[bigram[0]], freq / contextFreq[bigram[0]]]
        for bigram, freq in bigramFreq.items()
    ]

    return pd.DataFrame(bigramData, columns=['Term1', 'Term2', 'BigramFreq', 'ContextFreq', 'BigramProb']).sort_values(by='BigramFreq', ascending=False)

# Probabilidades de trigramas
def trigramProbabilities(tokenizedCorpus):
    trigrams = [ngram for text in tokenizedCorpus for ngram in generateNgrams(text, 3)]
    trigramFreq = Counter(trigrams)
    bigramContextFreq = Counter([trigram[:2] for trigram in trigrams])

    trigramData = [
        [trigram[0], trigram[1], trigram[2], freq, bigramContextFreq[trigram[:2]], freq / bigramContextFreq[trigram[:2]]]
        for trigram, freq in trigramFreq.items()
    ]

    return pd.DataFrame(trigramData, columns=['Term1', 'Term2', 'Term3', 'TrigramFreq', 'BigramContextFreq', 'TrigramProb']).sort_values(by='TrigramFreq', ascending=False)

# Guardar datos de n-gramas
def saveNgramData(bigramData, trigramData):
    # Ya están ordenados en las funciones anteriores, no es necesario volver a ordenar
    bigramData.to_csv('bigrams.csv', index=False)
    trigramData.to_csv('trigrams.csv', index=False)

# Función para cargar el archivo CSV
def loadFile():
    filePath = filedialog.askopenfilename(filetypes=[("CSV files", "*.csv")])
    if filePath:
        try:
            df = pd.read_csv(filePath)
            messages = df['Message'].tolist()
            tokenizedCorpus = tokenizeCorpus(messages)
            generateNgramFiles(tokenizedCorpus)
        except Exception as e:
            messagebox.showerror("Error", f"Error al cargar el archivo: {e}")

# Generar archivos de bigramas y trigramas
def generateNgramFiles(tokenizedCorpus):
    try:
        bigramas = bigramProbabilities(tokenizedCorpus)
        trigramas = trigramProbabilities(tokenizedCorpus)

        saveNgramData(bigramas, trigramas)

        # Mostrar resultados en la interfaz
        showNgrams(bigramas.head(10), trigramas.head(10))
        messagebox.showinfo("Éxito", "Los archivos 'bigrams.csv' y 'trigrams.csv' han sido generados.")
    except Exception as e:
        messagebox.showerror("Error", f"Error al generar N-gramas: {e}")

# Mostrar los N-gramas
def showNgrams(bigrams, trigrams):
    textBox.delete(1.0, tk.END)  # Limpiar la caja de texto
    textBox.insert(tk.END, "Bigramas (Top 10):\n")
    textBox.insert(tk.END, bigrams.to_string(index=False))
    textBox.insert(tk.END, "\n\nTrigramas (Top 10):\n")
    textBox.insert(tk.END, trigrams.to_string(index=False))

# Interfaz
root = tk.Tk()
root.title("Generador de N-gramas")
root.geometry("700x500")

# Frame principal
frame = ttk.Frame(root, padding="10")
frame.pack(fill="both", expand=True)

# Botones
loadButton = ttk.Button(frame, text="Cargar CSV", command=loadFile)
loadButton.pack(pady=10)

# Generar bigramas y trigramas
generateButton = ttk.Button(frame, text="Generar Bigramas y Trigramas", command=lambda: messagebox.showinfo("Info", "Primero debes seleccionar un archivo CSV."))
generateButton.pack(pady=10)

# Caja de resultados
textBox = tk.Text(frame, wrap="word", height=15)
textBox.pack(fill="both", expand=True, padx=10, pady=10)


In [6]:
root.mainloop()

2024-11-05 20:28:54.676 python[21503:1918150] +[IMKClient subclass]: chose IMKClient_Modern
2024-11-05 20:28:54.676 python[21503:1918150] +[IMKInputSession subclass]: chose IMKInputSession_Modern
2024-11-05 20:28:56.476 python[21503:1918150] The class 'NSOpenPanel' overrides the method identifier.  This method is implemented by class 'NSWindow'
